In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("all_kindle_review.csv")

In [3]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [4]:
data=data[['reviewText','rating']]

In [5]:
data

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [6]:
data.shape

(12000, 2)

In [7]:
data['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

Preprocessing AND Cleaning

In [8]:
data['rating']=data['rating'].apply(lambda X:0 if X<3 else 1)

In [9]:
data['rating'].nunique()

2

In [10]:
data['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [11]:
##Lower all Cases
data['reviewText']=data['reviewText'].str.lower()

In [12]:
data['reviewText'].head()

0    jace rankin may be short, but he's nothing to ...
1    great short read.  i didn't want to put it dow...
2    i'll start by saying this is the first of four...
3    aggie is angela lansbury who carries pocketboo...
4    i did not expect this type of book to be in li...
Name: reviewText, dtype: object

In [13]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from bs4 import BeautifulSoup

In [15]:
## Removing special characters
data['reviewText']=data['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))

In [16]:
## Remove the stopswords
data['reviewText']=data['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))

In [16]:
## Remove url 
data['reviewText']=data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))


In [17]:
## Remove html tags
data['reviewText']=data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

C:\Users\pc\AppData\Local\Temp\ipykernel_15816\1623791588.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data['reviewText']=data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())


In [18]:
## Remove any additional spaces
data['reviewText']=data['reviewText'].apply(lambda x: " ".join(x.split()))

In [19]:
data.head()

,reviewText,rating
0,jace rankin may be short but hes nothing to me...,1
1,great short read i didnt want to put it down s...,1
2,ill start by saying this is the first of four ...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [20]:
#Lemmatizer
from nltk.stem import WordNetLemmatizer

In [21]:
lemmatizer=WordNetLemmatizer()

In [22]:
def lemmatize_words (text):
    return " ".join([lemmatizer.lemmatize (word) for word in text.split()])

In [23]:
data['reviewText']=data['reviewText'].apply(lambda X:lemmatize_words(X))

Train-Test Split

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data['reviewText'],data['rating'],test_size=0.2)

Bag of WORDS

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform (X_train).toarray()
X_test_bow=bow.transform (X_test).toarray()

TF-IDF

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [36]:
len(X_train_tfidf[0])

36205

In [38]:
data['avg_word2vec'][0]

array([-0.09349847,  0.031362  ,  0.00837337,  0.07068944, -0.02934012,
       -0.0866156 ,  0.08911689,  0.10455872, -0.13516523, -0.12298634,
       -0.01059383, -0.09639332, -0.07039955,  0.01582122,  0.00341755,
        0.00490135,  0.0301031 ,  0.03056487, -0.11086498, -0.28520995,
        0.03797633,  0.05117733,  0.11624677,  0.01630511,  0.03015076,
        0.00322457, -0.09966027, -0.01469445, -0.05515676, -0.00521375,
        0.08151352, -0.04530986,  0.03834743, -0.0085862 ,  0.02378118,
        0.01987552,  0.07038628, -0.00662725,  0.03925635, -0.05698366,
       -0.00253448, -0.03755543, -0.01997285, -0.01011643,  0.01618567,
       -0.09939437, -0.01093594,  0.04809786,  0.05814897,  0.03511947,
       -0.0008084 , -0.02071055,  0.03531931,  0.02680468, -0.00243229,
        0.01681807,  0.07715809, -0.05006483, -0.0057736 ,  0.08752982,
        0.05683954,  0.02078168,  0.11038902, -0.01114771, -0.01369959,
        0.11015921,  0.0176374 ,  0.13271105, -0.1208325 , -0.00

Word2Vec

In [27]:
import gensim

In [28]:
from gensim.models import Word2Vec

In [29]:
sentences = X_train.tolist()
word2vec_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)

word2vec_model.save("word2vec.model")


In [31]:
import numpy as np

def avg_word2vec(sentence, model):
    words = sentence  
    word_vectors = [model.wv[word] for word in words if word in model.wv]

    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)  

    return np.mean(word_vectors, axis=0)

# Apply function to dataset
data["avg_word2vec"] = X_train.apply(lambda x: avg_word2vec(x, word2vec_model))



In [32]:
len(data['avg_word2vec'][0])

100

In [51]:
len(X_train_bow[0])

36156

In [31]:
X_train_tfidf[0]

array([0., 0., 0., ..., 0., 0., 0.])

Building Model

In [32]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow, y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [55]:
nb_model_wordd2Vecf=GaussianNB().fit(X_train_word2Vec,data['rating'])

Testing Model & Making Predictions

In [33]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [34]:
y_pred_bow=nb_model_bow.predict(X_test_bow)

In [35]:
y_pred_tfidf=nb_model_tfidf.predict(X_test_tfidf)

In [ ]:
y_pred_word2Vec=nb_model_wordd2Vecf.predict(X_test_tfidf)

In [36]:
confusion_matrix(y_test,y_pred_bow)

array([[535, 278],
       [727, 860]], dtype=int64)

In [37]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.58125


In [38]:
confusion_matrix(y_test,y_pred_tfidf)

array([[527, 286],
       [702, 885]], dtype=int64)

In [39]:
print("TF-IDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TF-IDF accuracy:  0.5883333333333334
